
Carmen Wall
Attachments
10:02 AM (21 minutes ago)
to me

hey honey, 
Given a product brief, return 3 starter recipes (ingredient list + inclusion %) plus a quick nutrition + cost summary.

Inputs (from the Brief_Template tab):

product type: dry kibble

life stage: adult / puppy

diet claim: grain-free / grain-inclusive / limited ingredient

main protein: chicken / lamb / salmon etc

price band: value / mid / premium

excludes (allergens)

targets: protein min, fat min (and if you want, moisture/fiber ranges)

Outputs 
For each of 3 options:

ingredient list + inclusion %

estimated protein %, fat %, fiber %, moisture %

estimated cost/kg

a short “why this” note (ex: closest historical recipes / what it matched on)

pass/fail flags for the constraints

Chatgpt did the synthetic data. 

#### Inputs:
- product type: e.g. dry kibble
- life stage: e.g. adult
- diet claim: e.g. grain_free
- main protein preference: e.g. lamb
- price band: e.g. premium
- excludes (allergens): e.g. corn, soy, wheat
- targets: e.g. protein min 28%, fat min 14%

#### Outputs:
### Recipe Options (3 total):
- **Ingredient List + Inclusion %:**
- **Estimated Nutrition + cost/kg:**
- **Short Why These explanation:**

#### Datasets:
- Ingredients (what's possible): ingredient_id, ingredient_name, category, protein_pct, fat_pct, fiber_pct, ash_pct, moisture_pct, cost_per_kg, allergens, max_inclusion_pct
- ingredient_lots (how stable it is): lot_id, ingredient_id, date, protein_pct, fat_pct, moisture_pct, supplier
- historical_recipes (what's proven): recipe_id, product_type, life_stage, diet_claim, main_protein, price_band, target_protein_min, target_fat_min, is_successful, achieved_protein_pct, achieved_fat_pct, achieved_fiber_pct, achieved_moisture_pct, estimated_cost_per_kg
- recipe_lines (how things were actually built): recipe_id, ingredient_id, inclusion_pct
- line_calc (math traceability): recipe_id, ingredient_id, inclusion_pct, ingredient_name, category, protein_pct, fat_pct, fiber_pct, moisture_pct, cost_per_kg, protein_contrib, fat_contrib, fiber_contrib, moisture_contrib, cost_contrib
- output_dataset (final results): option, recipe_id, estimated_protein_pct, estimated_fat_pct, estimated_moisture_pct, estimated_cost_per_kg, notes

## Import Data & Understand each set

In [1]:
import pandas as pd
import numpy as np

In [13]:
hist = pd.read_csv('C:\\Users\\justi\\DS_Projects\\Personal\\Portfolio_Projects\\data\\historical_recipes',sep='\t')
hist.head()

,recipe_id,product_type,life_stage,diet_claim,main_protein,price_band,target_protein_min,target_fat_min,is_successful,achieved_protein_pct,achieved_fat_pct,achieved_fiber_pct,achieved_moisture_pct,estimated_cost_per_kg
0,R001,dry_kibble,adult,limited_ingredient,Chicken,premium,24.0,14.0,0,21.6,47.4,20.9,2.3,$6.94
1,R002,dry_kibble,puppy,grain_free,Lamb,premium,28.0,16.0,1,19.6,37.9,25.7,2.8,$6.60
2,R003,dry_kibble,puppy,grain_free,Beef,value,28.0,16.0,1,19.2,32.8,31.5,7.3,$5.50
3,R004,dry_kibble,adult,limited_ingredient,Beef,mid,24.0,14.0,0,16.0,36.6,22.6,3.5,$6.59
4,R005,dry_kibble,adult,grain_free,Beef,mid,24.0,14.0,1,19.0,38.1,22.3,9.3,$6.22


In [12]:
lots = pd.read_csv('C:\\Users\\justi\\DS_Projects\\Personal\\Portfolio_Projects\\data\\ingredient_lots',sep='\t')
lots.head()

,lot_id,ingredient_id,date,protein_pct,fat_pct,moisture_pct,supplier
0,L0001,I001,2025-01-03,66.9,11.8,8.9,SUP_A
1,L0002,I001,2025-01-18,66.1,10.9,7.5,SUP_B
2,L0003,I001,2025-01-31,66.7,12.3,8.0,SUP_A
3,L0004,I001,2025-02-19,67.5,12.3,8.2,SUP_A
4,L0005,I001,2025-03-06,64.5,11.9,8.1,SUP_B


In [14]:
line = pd.read_csv('C:\\Users\\justi\\DS_Projects\\Personal\\Portfolio_Projects\\data\\line_calc',sep='\t')
line.head()

,recipe_id,ingredient_id,inclusion_pct,ingredient_name,category,protein_pct,fat_pct,fiber_pct,moisture_pct,cost_per_kg,protein_contrib,fat_contrib,fiber_contrib,moisture_contrib,cost_contrib
0,R001,I001,50.2,Chicken meal,animal_protein,65.0,12.0,1.5,8.0,$1.80,32.6,6.0,0.8,4.0,$0.90
1,R001,I003,23.5,Salmon meal,animal_protein,60.0,10.0,1.0,8.0,$2.70,14.1,2.4,0.2,1.9,$0.63
2,R001,I012,12.5,Canola oil,fat,0.0,100.0,0.0,0.2,$1.90,0.0,12.5,0.0,0.0,$0.24
3,R001,I013,7.4,Beet pulp,fiber,9.0,1.0,18.0,10.0,$0.70,0.7,0.1,1.3,0.7,$0.05
4,R001,I018,4.3,Natural flavor,other,0.0,0.0,0.0,5.0,$3.50,0.0,0.0,0.0,0.2,$0.15


In [15]:
recipes = pd.read_csv('C:\\Users\\justi\\DS_Projects\\Personal\\Portfolio_Projects\\data\\recipe_lines',sep='\t')
recipes.head()

,recipe_id,ingredient_id,inclusion_pct
0,R001,I001,50.2
1,R001,I003,23.5
2,R001,I012,12.5
3,R001,I013,7.4
4,R001,I018,4.3


In [61]:
ingredients = pd.read_csv('C:\\Users\\justi\\DS_Projects\\Personal\\Portfolio_Projects\\data\\ingredients',sep='\t')
ingredients['cost_per_kg'] = ingredients['cost_per_kg'].replace('[\$,]', '', regex=True).astype(float)
ingredients.head(50)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\justi\AppData\Local\Temp\ipykernel_25476\3917057072.py:2: SyntaxWarning: invalid escape sequence '\$'
  ingredients['cost_per_kg'] = ingredients['cost_per_kg'].replace('[\$,]', '', regex=True).astype(float)


,ingredient_id,ingredient_name,category,protein_pct,fat_pct,fiber_pct,ash_pct,moisture_pct,cost_per_kg,allergens,max_inclusion_pct
0,I001,Chicken meal,animal_protein,65.0,12.0,1.5,7.0,8.0,1.80,NaN,35.0
1,I002,Lamb meal,animal_protein,58.0,14.0,2.0,8.0,8.0,2.40,NaN,30.0
2,I003,Salmon meal,animal_protein,60.0,10.0,1.0,6.0,8.0,2.70,NaN,25.0
3,I004,Turkey meal,animal_protein,62.0,12.0,1.5,7.0,8.0,2.05,NaN,35.0
4,I005,Beef meal,animal_protein,55.0,13.0,2.0,9.0,8.0,2.10,NaN,25.0
5,I006,Pea protein,plant_protein,80.0,2.0,2.5,6.0,8.0,1.90,legume,20.0
6,I007,Soybean meal,plant_protein,48.0,1.5,3.5,6.0,11.0,0.75,soy,18.0
7,I008,Potato starch,starch,1.0,0.2,0.5,1.0,12.0,0.95,NaN,35.0
8,I009,Brown rice,starch,8.0,2.0,1.0,1.5,12.0,0.85,grain,35.0
9,I010,Corn meal,starch,9.0,3.5,2.0,1.5,12.0,0.65,"grain,corn",35.0


## Set up Optimization Problem
### Define Decision Variables
- for each ingredient i in Ingredients:
    - x_i = inclusion percentage of ingredient i in the recipe (continuous variable, 0 <= x_i <= max_inclusion_pct_i)
### Define Objective Function
- Minimize total cost per kg:
    - Minimize Z = Σ (cost_per_kg_i * x_i) for all ingredients i
### Define Constraints
- Nutritional Constraints:
- Exclude Allergens:
- Ingredient Inclusion Limits:
- Grain Free
- Main protein preference

In [36]:
import pulp as lp

In [56]:
def solve_one_recipe(ingredients_df, targets, rules):
    """
    ingredients_df
        ingredient_id | cost_per_kg | protein | fat | fiber | moisture | max_inclusion | is_allowed | is_main_protein
    targets
        dict of target nutrient values {"protein_min": 0.28, "fat_min": 0.14}
    Rules
        dict of rules {"main_protein_min": 0.20}
    """

    df = ingredients_df.copy()

    # Decision variables: inclusion fractions
    x = {
        row.ingredient_id: lp.LpVariable(
            f"x_{row.ingredient_id}", lowBound=0, upBound=row.max_inclusion_pct)
        for row in df.itertuples(index=False)
    }

    # LP Problem
    prob = lp.LpProblem("recipe_formulation", lp.LpMinimize)

    # Objective: minimize cost/kg
    prob += lp.lpSum(x[i] * df.loc[df["ingredient_id"] == i, "cost_per_kg"].iloc[0] for i in x)

    # constraint: sum to 1 (100%)
    prob += lp.lpSum(x[i] for i in x) == 1

    # hard rule: dissallowed ingredients must be zero
    for row in df.itertuples(index=False):
        if row.is_allowed == 0:
            prob += x[row.ingredient_id] == 0.0

    # nutrition mins
    prob += lp.lpSum(x[row.ingredient_id] * row.protein_pct for row in df.itertuples(index=False)) >= targets['protein_min']
    prob += lp.lpSum(x[row.ingredient_id] * row.fat_pct for row in df.itertuples(index=False)) >= targets['fat_min']

    # main protein minimum
    if "main_protein_min" in rules:
        prob += lp.lpSum(
            x[row.ingredient_id] for row in df.itertuples(index=False) if row.is_main_protein == 1
        ) >= rules["main_protein_min"]

    # Solve
    prob.solve(lp.PULP_CBC_CMD(msg=False))

    if lp.LpStatus[prob.status] != 'Optimal':
        return None, lp.LpStatus[prob.status]  # No feasible solution found

    # Extract solution
    sol = []

    for i, var in x.items():
        val = var.value()
        if val and val > 1e-6:
            sol.append((i,val))

    return sol, 'Optimal'

In [159]:
def summarize_solution(sol, ingredients_df):
    df = ingredients_df.set_index('ingredient_id')

    total_cost = 0
    totals = {"protein": 0, "fat": 0, "fiber": 0, "moisture": 0}
    for ing_id, frac in sol:
        row = df.loc[ing_id]
        total_cost += frac * row['cost_per_kg']
        totals['protein'] += frac * row['protein_pct']
        totals['fat'] += frac * row['fat_pct']
        totals['fiber'] += frac * row['fiber_pct']
        totals['moisture'] += frac * row['moisture_pct']

    ings_df = ingredients_df[['ingredient_id', 'ingredient_name']].set_index('ingredient_id')
    ings_dict = ings_df.to_dict()['ingredient_name']
    ingredients_dict = {ings_dict[ing_id]: frac for ing_id, frac in sol}

    return {
        "cost_per_kg": f"${total_cost:.2f}",
        "protein_pct": f"{totals['protein']:.2f}%",
        "fat_pct": f"{totals['fat']:.2f}%",
        "fiber_pct": f"{totals['fiber']:.2f}%",
        "moisture_pct": f"{totals['moisture']:.2f}%",
        "ingredients": ingredients_dict
    }

In [160]:
def get_top_ingredient(sol):
    return max(sol, key=lambda t: t[1])[0]

def solve_three_recipes(base_df, targets, rules):
    df = base_df.copy()
    recipes = []

    for k in range(3):
        sol, status = solve_one_recipe(df, targets, rules)
        if status != 'Optimal':
            break
        recipes.append(sol)

    # force diversity - ban top ingredient for next recipe
        top_ing = get_top_ingredient(sol)
        df.loc[df.ingredient_id == top_ing, 'is_allowed'] = 0

    return recipes

In [161]:
# Create a test case
ingredients_df = ingredients[['ingredient_id', 'cost_per_kg', 'protein_pct', 'fat_pct', 'fiber_pct', 'moisture_pct', 'max_inclusion_pct']].copy()
ingredients_df['is_allowed'] = 1
ingredients_df['is_main_protein'] = np.where(ingredients_df['ingredient_id'].isin(['I001']), 1, 0)  # Example main proteins
targets = {"protein_min": 0.28, "fat_min": 0.14}
rules = {"main_protein_min": 0.20}

In [162]:
test_recipes = solve_three_recipes(ingredients_df, targets, rules)

In [163]:
single_summary = summarize_solution(test_recipes[0], ingredients)
single_summary

{'cost_per_kg': '$0.44',
 'protein_pct': '13.00%',
 'fat_pct': '2.40%',
 'fiber_pct': '0.30%',
 'moisture_pct': '2.40%',
 'ingredients': {'Chicken meal': 0.2, 'Salt': 0.8}}